In [1]:
import camelot
import re

In [2]:
tables = camelot.read_pdf("pdfs/Boletim-Epidemiologico-COVID-19-2021.05.08.pdf", pages="18, 19, 20, 33, 34, 35, 36", strip_text="▼, ▲, \n")
len(tables)

# 18, 19, 20
# 33, 34, 35, 36

8

In [3]:
def add_new_row(data_frame, name_row, t_confirmado, t_obito):
    data_frame.loc[-1] = [name_row, t_confirmado, t_obito]  # adding a row
    data_frame.index = data_frame.index + 1  # shifting index
    return data_frame.sort_index()  # sorting by index

In [4]:
first_line = "MunicípioÓbitosDistribuiçãoLetalidadeMortalidade"
for table in tables:
    if table.df.at[0, 1] == "RT-PCR":
        table.df = table.df.filter([0,9])
        # Creat and appending table
        if not ('data_frame_confirmed' in vars() or 'data_frame_confirmed' in globals()):
            data_frame_confirmed = table.df
        else:
            data_frame_confirmed = data_frame_confirmed.append(table.df, ignore_index=True)


    elif table.df.at[0, 0] == first_line or table.df.at[0, 1] == first_line or table.df.at[0, 2] == first_line:
        table.df.drop(columns=[0, 4, 5, 6], inplace=True)
        table.df = table.df.drop([0]).reset_index()
        
        # Rows format
        for i in range(len(table.df[2])):
            if table.df.at[i, 1] != "=" and table.df.at[i, 1] != "":
                table.df.at[i, 2] = "".join(i for i in table.df.at[i, 1] if not i.isdigit())
        
        # Creat and appending table
        if not ('data_frame' in vars() or 'data_frame' in globals()):
            data_frame = table.df
        else:
            data_frame = data_frame.append(table.df, ignore_index=True)
        
        data_frame.drop(columns=[1, "index"], inplace=True)

# Remove last row
data_frame_confirmed.drop(index=data_frame_confirmed.index[-1], 
        axis=0, 
        inplace=True)
# Remove first line
data_frame_confirmed.drop([0], inplace=True)
confirmed_col = data_frame_confirmed[9]

data_frame.rename(columns={2: "municipio", 3: "mortes"}, inplace=True)
for i in range(len(data_frame["municipio"])):
    data_frame.at[i, "municipio"] = " ".join(re.findall('[A-ZÀ-Ú][a-zà-ú]*', data_frame.at[i, "municipio"]))
data_frame.sort_values(by=['municipio'], ascending=True, inplace=True)
add_new_row

# Inserting the column at the
# beginning in the DataFrame
data_frame.insert(loc = 1,
          column = 'confirmados',
          value = confirmed_col)
display(data_frame)
data_frame.to_csv("test.csv", line_terminator=None, index=False)

SyntaxError: invalid syntax (<ipython-input-4-8ef0b33ecfd9>, line 12)